In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [2]:
base_model = VGG16(weights='imagenet', include_top=False)

In [5]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, activation='softmax'))

In [6]:
model.load_weights("../models/weight.hdf5")

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [9]:
df = pd.DataFrame()
vid = []
label = []
for folder in os.listdir('../test'):
    for file in os.listdir('../test/' + folder):
        label.append(folder)
        vid.append(file)
df['video'] = vid
df['label'] = label

In [13]:
test_videos = df['video']

In [41]:
# creating the tags
train = pd.read_csv('../train_new.csv')
y = train['class']
y = pd.get_dummies(y

In [20]:
# creating two lists to store predicted and actual tags



# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('../test/'+df['label'][i]+"/"+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='../temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    


100%|██████████| 2/2 [00:38<00:00, 19.09s/it]


In [119]:
prediction_images = []
actual = []
for folder in os.listdir('../temp'):
    actual.append('Groenling')
    img = image.load_img('../temp/' + folder, target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    prediction_images.append(img)



In [120]:
predict = []       
    # converting all the frames for a test video into numpy array
prediction_images = np.array(prediction_images)
prediction_images2 = np.array(prediction_images)
    # extracting features using pre-trained model
prediction_images = base_model.predict(prediction_images)

#     # converting features in one dimensional array
# prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
#     # predicting tags for each array
# prediction = model.predict(prediction_images)
#     # appending the mode of predictions in predict list to assign the tag to the video
# predict.append(y.columns.values[s.mode(prediction)[0][0]])


In [121]:
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)

In [122]:
prediction = model.predict(prediction_images)

In [123]:
classes_x=np.argmax(prediction,axis=1)



In [124]:
l = []
for name in os.listdir('../training'):
    l.append(name)

In [125]:
for x in classes_x:
    predict.append(l[x])

In [126]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

0.22075055187637968